**<font color = black size=6>实验六:随机森林</font>**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

**<font color = blue size=4>第一部分:函数介绍</font>**

介绍一些可能会用到的函数。

In [9]:
# np.random.choice函数从一个一维数组中随机采样
x = np.array([1,2,3,4])
y = np.random.choice(x, replace=True, size=10)
print(y)

# np.random.shuffle函数对一个数组/矩阵按照第一维进行洗牌
x = np.array([[0,1,2],[3,4,5],[6,7,8],[9,10,11],[12,13,14]])
np.random.shuffle(x)
print(x)

# DataFrame对象的sample函数可以随机采样n个数据或者采样比例为frac的数据
x = np.array([[0,0,0],[1,1,1],[2,2,2],[3,3,3],[4,4,4]])
frame = pd.DataFrame(x)
print(frame.sample(n=5))
print(frame.sample(frac=0.3))

[4 3 1 1 2 1 3 3 3 4]
[[ 9 10 11]
 [ 3  4  5]
 [ 0  1  2]
 [12 13 14]
 [ 6  7  8]]
   0  1  2
1  1  1  1
0  0  0  0
4  4  4  4
2  2  2  2
3  3  3  3
   0  1  2
2  2  2  2
0  0  0  0


**<font color = blue size=4>第二部分:实验任务</font>**

本次实验承接上次实验，实现随机森林。

<span style="color:purple">本次实验依旧使用泰坦尼克号数据集(train_titanic.csv, test_titanic.csv。数据集包括了四个属性特征以及一个标签(即为Survived,代表是否生还),属性特征分别为Sex(性别)，sibsp(堂兄弟妹个数)，Parch(父母与小孩的个数)，Pclass(乘客等级)  
其中该数据集无缺失值和异常值，且所有连续变量已自动转换为离散变量,标签(Survived)也自动转变为离散变量0和1，所以你无需进行数据预处理，可以直接使用该数据集。</span>

<span style="color:purple">1) 对上次实验的best_split函数进行修改，实现随机特征选择。  
先从特征集$A$中先随机选取$k$个特征构成特征集$A'$，再从$A'$中选取最佳划分的特征。$k$一般取$max\{log_2 d,1\}$, $d$是$A$的元素的个数。你可使用特征的信息增益来决定最佳划分的特征。  
    【输入】：数据集D、特征集A    
    【输出】：随机特征集A'中最佳划分的特征维数   
    【信息增益公式】:  
        某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征feature,该特征包含V个不同的取值，特征值为第v(v=1,2,...,V)个值的数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$  
    【信息熵公式】:  
        某数组包含K个不同的取值，样本为第k(k=1,2,...,K)个值的数量所占比例为p_k,则其信息熵为$$Ent=-\sum_{k=1}^K p_k log_2 p_k$$
</span>

In [10]:
def entropy(label):
    unique_labels, label_counts = np.unique(label, return_counts=True)
    total_samples = len(label)
    ent = 0

    for count in label_counts:
        probability = count / total_samples
        ent -= probability * np.log2(probability)

    return ent

def best_split(D, A):
    d = len(A)
    k = max(np.log2(d), 1)
    A_prime = np.random.choice(list(A), int(k), replace=False)

    def split_by_value(feature, label, value):
        indices = np.where(feature == value)
        split_label = label[indices]

        return split_label

    best_information_gain = 0
    best_dimension = None

    for dimension in A_prime:
        feature_values = D[:, dimension]
        unique_values = np.unique(feature_values)
        information_gain = entropy(
            D[:, -1]
        )  # Initialize with the entropy of the whole dataset

        for value in unique_values:
            split_label = split_by_value(feature_values, D[:, -1], value)
            information_gain -= (len(split_label) / len(D)) * entropy(split_label)

        if information_gain > best_information_gain:
            best_information_gain = information_gain
            best_dimension = dimension

    return best_dimension

<span style="color:purple">2) 对上次实验完成的决策树类进行修改。你需要实现下面三个函数：  
1. TreeGenerate(self, D, A)：递归构建决策树，伪代码参照提供的“Algorithm 1 决策树学习基本算法”。  
2. train(self, D)：做一些数据预处理，包括将Dataframe转换为numpy矩阵，从数据集中提取属性集，并调用TreeGenerate函数来递归地生成决策树。  
3. predict(self, D)：对测试集D进行预测，要求返回数据集D的预测标签，即一个(|D|,1)矩阵（|D|行1列）。  
由于训练集是采样生成，因此需要对predict函数做修改。需要考虑测试集中出现决策树无法划分的特征值时的情况。给出两种参考的做法：  
a).对其不再进行预测，直接给定划分失败的样本标签(例如-1)。  
b).跳过该划分节点，随机选取一个特征值继续遍历。</span>

In [11]:
# 记下所有属性可能的取值
train_frame = pd.read_csv('train_titanic.csv')
D = np.array(train_frame)
A = set(range(D.shape[1] - 1))
possible_value = {}
for every in A:
    possible_value[every] = np.unique(D[:, every])

In [12]:
# 树结点类
class Node:
    def __init__(self, isLeaf=True, label=-1, index=-1):
        self.isLeaf = isLeaf # isLeaf表示该结点是否是叶结点
        self.label = label # label表示该叶结点的label（当结点为叶结点时有用）
        self.index = index # index表示该分支结点的划分属性的序号（当结点为分支结点时有用）
        self.children = {} # children表示该结点的所有孩子结点，dict类型，方便进行决策树的搜索
        
    def addNode(self, val, node):
        self.children[val] = node #为当前结点增加一个划分属性的值为val的孩子结点

In [13]:
# 决策树类
class DTree:
    def __init__(self):
        self.tree_root = None  # 决策树的根结点
        self.possible_value = possible_value  # 用于存储每个特征可能的取值

    """
    TreeGenerate函数用于递归构建决策树，伪代码参照课件中的“Algorithm 1 决策树学习基本算法”
     
    """

    def TreeGenerate(self, D, A):
        # 生成结点 node
        node = Node()

        # if D中样本全属于同一类别C then
        #     将node标记为C类叶结点并返回
        # end if
        if len(np.unique(D[:, -1])) == 1:
            node.isLeaf = True
            node.label = D[0, -1]
            return node

        # if A = Ø OR D中样本在A上取值相同 then
        #     将node标记叶结点，其类别标记为D中样本数最多的类并返回
        # end if
        tmp = np.array(list(A))
        if len(tmp) == 0 or len(np.unique(D[:, tmp])) == 1:
            node.isLeaf = True
            node.label = Counter(D[:, -1]).most_common(1)[0][0]
            return node

        # 从A中选择最优划分特征a_star
        # （选择信息增益最大的特征，用到上面实现的best_split函数）
        a_star = best_split(D, A)

        # for a_star 的每一个值a_star_v do
        #     为node 生成每一个分支；令D_v表示D中在a_star上取值为a_star_v的样本子集
        #     if D_v 为空 then
        #         将分支结点标记为叶结点，其类别标记为D中样本最多的类
        #     else
        #         以TreeGenerate(D_v,A-{a_star}) 为分支结点
        #     end if
        # end for
        # print("a_star:", a_star)
        if a_star is not None:
            for a_star_v in np.unique(D[:, a_star]):
                D_v = D[D[:, a_star] == a_star_v]
                if len(D_v) == 0:
                    node.addNode(
                        a_star_v, Node(True, Counter(D[:, -1]).most_common(1)[0][0])
                    )
                else:
                    node.addNode(
                        a_star_v,
                        self.TreeGenerate(
                            D[D[:, a_star] == a_star_v], A - {a_star}
                        ),  # 递归调用TreeGenerate函数
                    )

        # def __init__(self, isLeaf=True, label=-1, feature_index=-1) node的构造函数中，isLeaf默认为True，feature_index默认为-1。而函数执行到这里时，node的值需要赋值为False，feature_index需要赋值为a_star
            node.isLeaf = False
            node.feature_index = a_star
            return node
        else:
            node.isLeaf = True
            node.label = Counter(D[:, -1]).most_common(1)[0][0]
            return node

    """
    train函数可以做一些数据预处理（比如Dataframe到numpy矩阵的转换，提取属性集等），并调用TreeGenerate函数来递归地生成决策树
 
    """

    def train(self, D):
        D = np.array(D)  # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        
        A = set(range(D.shape[1] - 1))  # 特征集A
        # print(A)

        self.tree_root = self.TreeGenerate(D, A)  # 递归地生成决策树，并将决策树的根结点赋值给self.tree_root
        return

    """
    predict函数对测试集D进行预测， 并输出预测准确率（预测正确的个数 / 总数据数量）
    
    """

    def predict(self, D):
        D = np.array(D)  # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        # 对于D中的每一行数据d，从当前结点x=self.tree_root开始，当当前结点x为分支结点时，
        # 则搜索x的划分特征为该行数据相应的特征值的孩子结点（即x=x.children[d[x.index]]），不断重复，
        # 直至搜索到叶结点，该叶结点的标签就是数据d的预测标签
        label = []
        for d in D:
            x = self.tree_root
            succeed = True
            while not x.isLeaf:
                # print("x.feature_index:", x.feature_index)
                if d[x.feature_index] not in x.children:
                    succeed = False
                    break
                x = x.children[d[x.feature_index]]
            # print(x.label)
            if succeed:
                label.append(x.label)
            else:
                # 对其不再进行预测，直接给定划分失败的样本标签为-1
                label.append(-1)
        return label

<span style="color:purple">3) 重复采用Bootstrap自助采样法对训练数据集'test_titanic.csv'进行采样，生成$n$个子训练数据集($n$自行设定)。  
Bootstrap采样法是指，每次从原数据集中【有放回】地随机采样一个样本，重复进行$m$次，就生成一个有$m$个样本的子数据集。</span>

In [14]:
train_frame = pd.read_csv('train_titanic.csv')

# Bootstrap 采样
n = 10
tree = [DTree()] * n
for i in range(n):
    D = np.array(train_frame)
    D = D[np.random.choice(D.shape[0], D.shape[0], replace=True)]
    D = pd.DataFrame(D)
    tree[i].train(D)



NameError: name 'Counter' is not defined

<span style="color:purple">4) 生成n棵决策树实例，使用上述生成的n个子训练数据集各自训练一棵决策树，即子训练集D1训练决策树1，子训练集D2训练决策树2……</span>

In [ ]:
# ----- Your code here -------





<span style="color:purple">5) 用训练完成的$n$棵决策树分别对测试数据集'test_titanic.csv'进行预测。采用相对多数投票法来对各棵决策树的预测结果进行结合。输出结合的预测结果的准确率。  
【相对多数投票法】  
对于某个样本$x$, 相对多数投票法预测它的标签为得票最多的标签。若同时有多个标签获得最高票，则从中随机选取一个。其公式如下所示：
$$H(x)=C_{\mathop{\arg\max}_{j} \sum_{i=1}^n h_i^j(x)}$$  
</span>

In [15]:
test_frame = pd.read_csv('test_titanic.csv')

# ----- Your code here -------
result = []
for t in tree:
    result.append(t.predict(test_frame))

# 相对多数投票
result = np.array(result)
res = []
for i in range(len(result[0])):
    res.append(Counter(result[:, i]).most_common(1)[0][0])

# print(res)

accuracy = np.sum(res == test_frame['Survived']) / len(test_frame)

print("Bootstrap采样的准确率为：", accuracy)



AttributeError: 'NoneType' object has no attribute 'isLeaf'

**<font color = blue size=4>第三部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)不要提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可

二、实验报告提交截止日期为：【10月27日 14:20】  
提交地址：https://send2me.cn/g_kfMtFI/SuiqyPO6B7rxqg  
要求：  
1)文件格式为：学号-姓名-实验六.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  

三、课堂课件获取地址: https://www.jianguoyun.com/p/DTGgCYAQp5WhChir26EFIAA  
实验内容获取地址: https://www.jianguoyun.com/p/DekWAFoQp5WhChis26EFIAA